# Vegetation Vigor Index

In [1]:
import os
import ee
import geemap
import rasterio
import datetime
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from shapely.geometry import MultiPoint
from dotenv import load_dotenv

NB: In order to run this script, you need to have the Earth Engine API initialised in your environment and also have a project created in the Google Cloud Platform with the Earth Engine API enabled. Its name is stored in the .env file as `EARTHENGINE_PROJECT_NAME`.

In [2]:
# Initialization of variables
load_dotenv()
ee.Initialize(project=os.getenv("EARTHENGINE_PROJECT_NAME"))

Let's load the yield data and create a region from the relevant points. It will be a convex hull of the valid yield data points.

In [3]:
with rasterio.open('Data/interpolated_yield_4326.tif') as src:
    interpolated_yield = src.read(1)  # Assuming it's a single band raster
    
    # Create a mask for non-NaN and non-nodata values
    yield_mask = (interpolated_yield != src.nodata) & (~np.isnan(interpolated_yield))
    
    # Get coordinates of non-NaN points
    rows, cols = np.where(yield_mask)
    coords = [src.xy(row, col) for row, col in zip(rows, cols)]

    # Get the exact bounds and resolution from the yield data
    yield_bounds = src.bounds
    yield_shape = src.shape
    resolution = src.res
    yield_transform = src.transform
    yield_crs = src.crs

# Create a convex hull of all the non-NaN yield data points
yield_hull = MultiPoint(coords).convex_hull

# Add a buffer to the convex hull
buffer_size = 5 / 111320  # Adjust this value as needed (in degrees)
yield_hull_buffered = yield_hull.buffer(buffer_size)

# Convert the buffered convex hull to an Earth Engine geometry
aoi_coords = list(yield_hull_buffered.exterior.coords)
aoi = ee.Geometry.Polygon(aoi_coords)

In [4]:
# Function to calculate GCVI
def calculateGCVI(image):
    gcvi = image.expression(
        '(NIR / GREEN) - 1',
        {
            'NIR': image.select('B8'),
            'GREEN': image.select('B3')
        }
    ).rename('GCVI')
    return image.addBands(gcvi)

# Function to calculate NDVI
def calculateNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [5]:
# Set the fixed end date to August 31, 2022
end_date = datetime.datetime(2022, 8, 31)
start_date = end_date - datetime.timedelta(days=365)

# Get Sentinel-2 imagery for the last year
collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(aoi)
              .filterDate(ee.Date(start_date.strftime('%Y-%m-%d')), ee.Date(end_date.strftime('%Y-%m-%d')))
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
              .map(calculateGCVI))

# Calculate average Vegetation vigor index
avg_index = collection.select('GCVI').mean()

The idea would be to retrieve the collection of images focused only on the aoi (area of interest) as defined above.

In [6]:
# Use the original yield bounds for exporting the NDVI tif
export_region = ee.Geometry.Rectangle(list(yield_bounds))

# Export the image locally using geemap
geemap.ee_export_image(
    avg_index,
    filename='Data/average_ndvi.tif',
    region=export_region,
    crs='EPSG:4326',
    crs_transform=[resolution[0], 0, yield_bounds.left, 0, -resolution[1], yield_bounds.top],
    #dimensions=[yield_shape[0], yield_shape[1]],
    file_per_band=False,
)

Generating URL ...
Please wait ...
Data downloaded to /Users/carlonestor/Documents/GitHub/sunflower_yield_analysis/Data/average_ndvi.tif


In [7]:
# Function to print raster metadata
def print_raster_info(filepath):
    with rasterio.open(filepath) as src:
        print(f"Metadata for {filepath}:")
        print(f"CRS: {src.crs}")
        print(f"Resolution: {src.res}")
        print(f"Bounds: {src.bounds}")
        print(f"Shape: {src.shape}")
        print("\n")

# Check metadata for both files
print_raster_info('Data/interpolated_yield_4326.tif')
print_raster_info('Data/average_ndvi.tif')

Metadata for Data/interpolated_yield_4326.tif:
CRS: EPSG:4326
Resolution: (8.983111749910169e-05, 8.983111749910169e-05)
Bounds: BoundingBox(left=11.964872146635857, bottom=44.847339020977785, right=11.983197694605673, top=44.85102209679525)
Shape: (41, 204)


Metadata for Data/average_ndvi.tif:
CRS: EPSG:4326
Resolution: (8.983111749910169e-05, 8.983111749910169e-05)
Bounds: BoundingBox(left=11.964872146635857, bottom=44.847249189860285, right=11.983287525723172, top=44.85102209679525)
Shape: (42, 205)




The first problem is that the two pictures in .tif must have the same resolution, shape, crs to be used together. As we can see above, this is not the case. There are some slight differences that needs to be taken care of.

The differences can be re-aligned by force, but the underlying real issue is that the images collected from the satellite show a different area (a polygon or rectangle rather than the triangular field). Hence, also the following correlation analysis is not correct.

For sure the problem lies in the inconsistency of the two images. We tried to solve it by having them in the same format, but it did not work.

In [15]:
# Load both datasets
with rasterio.open('Data/interpolated_yield_4326.tif') as yield_src, \
     rasterio.open('Data/average_ndvi.tif') as ndvi_src:
    
    yield_data = yield_src.read(1)
    ndvi_data = ndvi_src.read(1)
    
    # Check if shapes match
    if yield_data.shape != ndvi_data.shape:
        raise ValueError(f"Shape mismatch: Yield data shape {yield_data.shape} does not match NDVI data shape {ndvi_data.shape}")
    
    print(f"Data shapes match: {yield_data.shape}")
    
    # Flatten the arrays
    yield_valid = yield_data.flatten()
    ndvi_valid = ndvi_data.flatten()

Data shapes match: (40, 203)


In [ ]:
# Calculate correlation
correlation, p_value = stats.pearsonr(yield_valid, ndvi_valid)
print(f"\nCorrelation coefficient: {correlation}")
print(f"P-value: {p_value}")

# Create a scatter plot
plt.figure(figsize=(10, 8))
plt.scatter(ndvi_valid, yield_valid, alpha=0.1)
plt.xlabel('Average NDVI')
plt.ylabel('Yield')
plt.title('Yield vs Average NDVI')
plt.show()